**1. Predictions using Neural Network**

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from google.colab import drive
from IPython.display import display, HTML
import joblib
import os

# Mount Google Drive
drive.mount('/content/drive')

# Apply CSS styling
def apply_css():
    display(HTML("""
    <style>
        .main-header { color: #0066cc; text-align: center; font-size: 32px; font-weight: bold; margin-bottom: 20px; }
        .sub-header { color: #0066cc; font-size: 20px; font-weight: bold; margin: 20px 0 10px; }
        .prediction-box { background-color: #e6f0ff; border-radius: 10px; padding: 15px; margin: 10px 0; }
        .prediction-result { font-size: 20px; font-weight: bold; color: #006600; text-align: center; }
    </style>
    """))

# Load model and scaler
drive_path = '/content/drive/MyDrive/IPL match winner models'
model_path = os.path.join(drive_path, 'neural_network_model.h5')
scaler_path = os.path.join(drive_path, 'scaler.joblib')

if os.path.exists(model_path) and os.path.exists(scaler_path):
    model = load_model(model_path)
    scaler = joblib.load(scaler_path)
    print("Neural Network model and scaler loaded.")
    print("Expected features:", scaler.feature_names_in_)
else:
    print("Model or scaler not found. Please train the model first.")
    model, scaler = None, None

# Define teams and cities (adjust based on training data)
IPL_TEAMS = ['Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants',
             'Mumbai Indians', 'Chennai Super Kings', 'Royal Challengers Bangalore',
             'Sunrisers Hyderabad', 'Punjab Kings', 'Rajasthan Royals']
IPL_CITIES = ['Bengaluru', 'Chennai', 'Delhi', 'Dharamsala', 'Guwahati', 'Mumbai', 'Kolkata',
              'Hyderabad', 'Jaipur', 'Mullanpur/New Chandigarh', 'Visakhapatnam']

# Prepare input data
def prepare_input_data(runs_left, balls_left, wickets_remaining, total_run_x, crr, rrr, batting_team, bowling_team, city, scaler):
    data = {'runs_left': [runs_left], 'balls_left': [balls_left], 'wickets_remaining': [wickets_remaining],
            'total_run_x': [total_run_x], 'crr': [crr], 'rrr': [rrr]}
    input_df = pd.DataFrame(data)

    for team in IPL_TEAMS:
        input_df[f'batting_team_{team}'] = [1 if team == batting_team else 0]
        input_df[f'bowling_team_{team}'] = [1 if team == bowling_team else 0]
    for city_name in IPL_CITIES:
        input_df[f'city_{city_name}'] = [1 if city_name == city else 0]

    expected_features = scaler.feature_names_in_
    missing_features = [feat for feat in expected_features if feat not in input_df.columns]
    for feat in missing_features:
        input_df[feat] = 0
    input_df = input_df[expected_features]
    return input_df

# Main interface
def neural_network_interface():
    apply_css()
    display(HTML('<h1 class="main-header">Neural Network IPL Predictor</h1>'))
    if not model or not scaler:
        print("Cannot proceed without model and scaler.")
        return

    # Inputs
    display(HTML('<h2 class="sub-header">Match Situation</h2>'))
    runs_left = float(input("Runs Left (0-300): ") or 100)
    balls_left = float(input("Balls Left (0-120): ") or 60)
    wickets_remaining = int(input("Wickets Remaining (0-10): ") or 7)
    total_run_x = float(input("Target Score (0-300): ") or 180)

    current_score = total_run_x - runs_left
    overs_completed = 20 - (balls_left / 6)
    crr = current_score / overs_completed if overs_completed > 0 else 0
    rrr = (runs_left * 6) / balls_left if balls_left > 0 else 0
    print(f"\nCurrent Run Rate: {crr:.2f}")
    print(f"Required Run Rate: {rrr:.2f}")

    display(HTML('<h2 class="sub-header">Select Teams and City</h2>'))
    print(f"Teams: {IPL_TEAMS}")
    batting_team = input("Batting Team: ") or IPL_TEAMS[0]
    bowling_team = input("Bowling Team: ") or IPL_TEAMS[1]
    print(f"Cities: {IPL_CITIES}")
    city = input("City: ") or IPL_CITIES[0]

    if batting_team not in IPL_TEAMS or bowling_team not in IPL_TEAMS or city not in IPL_CITIES:
        print("Invalid input. Using defaults.")
        batting_team, bowling_team, city = IPL_TEAMS[0], IPL_TEAMS[1], IPL_CITIES[0]
    if batting_team == bowling_team:
        bowling_team = IPL_TEAMS[1] if batting_team == IPL_TEAMS[0] else IPL_TEAMS[0]

    # Predict
    input_data = prepare_input_data(runs_left, balls_left, wickets_remaining, total_run_x, crr, rrr, batting_team, bowling_team, city, scaler)
    input_scaled = scaler.transform(input_data)
    probability = model.predict(input_scaled, verbose=0)[0][0]
    prediction = 1 if probability > 0.5 else 0

    # Display result
    display(HTML('<div class="prediction-box">'))
    if prediction == 1:
        display(HTML(f'<p class="prediction-result">{batting_team} wins with {probability*100:.2f}% probability</p>'))
    else:
        display(HTML(f'<p class="prediction-result">{bowling_team} wins with {(1-probability)*100:.2f}% probability</p>'))
    print(f"Match Summary: {batting_team} vs {bowling_team} at {city}")
    print(f"Target: {total_run_x}, Score: {current_score}/{10-wickets_remaining}, Overs: {20-overs_completed:.1f}/20")
    display(HTML('</div>'))

if __name__ == '__main__':
    neural_network_interface()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Neural Network model and scaler loaded.
Expected features: ['runs_left' 'balls_left' 'wickets_remaining' 'total_run_x' 'crr' 'rrr'
 'batting_team_Delhi Capitals' 'batting_team_Gujarat Titans'
 'batting_team_Kolkata Knight Riders' 'batting_team_Lucknow Super Giants'
 'batting_team_Mumbai Indians' 'batting_team_Punjab Kings'
 'batting_team_Rajasthan Royals'
 'batting_team_Royal Challengers Bangalore'
 'batting_team_Sunrisers Hyderabad' 'bowling_team_Delhi Capitals'
 'bowling_team_Gujarat Titans' 'bowling_team_Kolkata Knight Riders'
 'bowling_team_Lucknow Super Giants' 'bowling_team_Mumbai Indians'
 'bowling_team_Punjab Kings' 'bowling_team_Rajasthan Royals'
 'bowling_team_Royal Challengers Bangalore'
 'bowling_team_Sunrisers Hyderabad' 'city_Bengaluru' 'city_Chennai'
 'city_Delhi' 'city_Dharamsala' 'city_Guwahati' 'city_Hyderabad'
 'city_Jaipur' 'city_Kolkata' 

Runs Left (0-300): 100
Balls Left (0-120): 55
Wickets Remaining (0-10): 5
Target Score (0-300): 212

Current Run Rate: 10.34
Required Run Rate: 10.91


Teams: ['Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians', 'Chennai Super Kings', 'Royal Challengers Bangalore', 'Sunrisers Hyderabad', 'Punjab Kings', 'Rajasthan Royals']
Batting Team: Mumbai Indians
Bowling Team: Chennai Super Kings
Cities: ['Bengaluru', 'Chennai', 'Delhi', 'Dharamsala', 'Guwahati', 'Mumbai', 'Kolkata', 'Hyderabad', 'Jaipur', 'Mullanpur/New Chandigarh', 'Visakhapatnam']
City: Chennai


Match Summary: Mumbai Indians vs Chennai Super Kings at Chennai
Target: 212.0, Score: 112.0/5, Overs: 9.2/20


*The model’s raw output probabilities might not be well-calibrated. Neural Networks with a sigmoid output (like yours) can produce extreme values (e.g., 0.9999 or 0.0001), which get rounded to 100% or 0%.*

**2. Predictions using random forest**

In [ ]:
import pandas as pd
import numpy as np
import joblib
from google.colab import drive
from IPython.display import display, HTML
import os

# Mount Google Drive
drive.mount('/content/drive')

# Apply CSS styling
def apply_css():
    display(HTML("""
    <style>
        .main-header { color: #0066cc; text-align: center; font-size: 32px; font-weight: bold; margin-bottom: 20px; }
        .sub-header { color: #0066cc; font-size: 20px; font-weight: bold; margin: 20px 0 10px; }
        .prediction-box { background-color: #e6f0ff; border-radius: 10px; padding: 15px; margin: 10px 0; }
        .prediction-result { font-size: 20px; font-weight: bold; color: #006600; text-align: center; }
    </style>
    """))

# Load model and scaler
drive_path = '/content/drive/MyDrive/IPL match winner models'
model_path = os.path.join(drive_path, 'random_forest_model.pkl')
scaler_path = os.path.join(drive_path, 'scaler.joblib')

if os.path.exists(model_path) and os.path.exists(scaler_path):
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print("Random Forest model and scaler loaded.")
    print("Expected features:", scaler.feature_names_in_)
else:
    print("Model or scaler not found. Please train the model first.")
    model, scaler = None, None

# Define teams and cities
IPL_TEAMS = ['Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants',
             'Mumbai Indians', 'Chennai Super Kings', 'Royal Challengers Bangalore',
             'Sunrisers Hyderabad', 'Punjab Kings', 'Rajasthan Royals']
IPL_CITIES = ['Bengaluru', 'Chennai', 'Delhi', 'Dharamsala', 'Guwahati', 'Mumbai', 'Kolkata',
              'Hyderabad', 'Jaipur', 'Mullanpur/New Chandigarh', 'Visakhapatnam']

# Prepare input data
def prepare_input_data(runs_left, balls_left, wickets_remaining, total_run_x, crr, rrr, batting_team, bowling_team, city, scaler):
    data = {'runs_left': [runs_left], 'balls_left': [balls_left], 'wickets_remaining': [wickets_remaining],
            'total_run_x': [total_run_x], 'crr': [crr], 'rrr': [rrr]}
    input_df = pd.DataFrame(data)

    for team in IPL_TEAMS:
        input_df[f'batting_team_{team}'] = [1 if team == batting_team else 0]
        input_df[f'bowling_team_{team}'] = [1 if team == bowling_team else 0]
    for city_name in IPL_CITIES:
        input_df[f'city_{city_name}'] = [1 if city_name == city else 0]

    expected_features = scaler.feature_names_in_
    missing_features = [feat for feat in expected_features if feat not in input_df.columns]
    for feat in missing_features:
        input_df[feat] = 0
    input_df = input_df[expected_features]
    return input_df

# Main interface
def random_forest_interface():
    apply_css()
    display(HTML('<h1 class="main-header">Random Forest IPL Predictor</h1>'))
    if not model or not scaler:
        print("Cannot proceed without model and scaler.")
        return

    # Inputs
    display(HTML('<h2 class="sub-header">Match Situation</h2>'))
    runs_left = float(input("Runs Left (0-300): ") or 100)
    balls_left = float(input("Balls Left (0-120): ") or 60)
    wickets_remaining = int(input("Wickets Remaining (0-10): ") or 7)
    total_run_x = float(input("Target Score (0-300): ") or 180)

    current_score = total_run_x - runs_left
    overs_completed = 20 - (balls_left / 6)
    crr = current_score / overs_completed if overs_completed > 0 else 0
    rrr = (runs_left * 6) / balls_left if balls_left > 0 else 0
    print(f"\nCurrent Run Rate: {crr:.2f}")
    print(f"Required Run Rate: {rrr:.2f}")

    display(HTML('<h2 class="sub-header">Select Teams and City</h2>'))
    print(f"Teams: {IPL_TEAMS}")
    batting_team = input("Batting Team: ") or IPL_TEAMS[0]
    bowling_team = input("Bowling Team: ") or IPL_TEAMS[1]
    print(f"Cities: {IPL_CITIES}")
    city = input("City: ") or IPL_CITIES[0]

    if batting_team not in IPL_TEAMS or bowling_team not in IPL_TEAMS or city not in IPL_CITIES:
        print("Invalid input. Using defaults.")
        batting_team, bowling_team, city = IPL_TEAMS[0], IPL_TEAMS[1], IPL_CITIES[0]
    if batting_team == bowling_team:
        bowling_team = IPL_TEAMS[1] if batting_team == IPL_TEAMS[0] else IPL_TEAMS[0]

    # Predict
    input_data = prepare_input_data(runs_left, balls_left, wickets_remaining, total_run_x, crr, rrr, batting_team, bowling_team, city, scaler)
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]
    probability = model.predict_proba(input_scaled)[0][1] if prediction == 1 else model.predict_proba(input_scaled)[0][0]

    # Display result
    display(HTML('<div class="prediction-box">'))
    if prediction == 1:
        display(HTML(f'<p class="prediction-result">{batting_team} wins with {probability*100:.2f}% probability</p>'))
    else:
        display(HTML(f'<p class="prediction-result">{bowling_team} wins with {(1-probability)*100:.2f}% probability</p>'))
    print(f"Match Summary: {batting_team} vs {bowling_team} at {city}")
    print(f"Target: {total_run_x}, Score: {current_score}/{10-wickets_remaining}, Overs: {20-overs_completed:.1f}/20")
    display(HTML('</div>'))

if __name__ == '__main__':
    random_forest_interface()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Random Forest model and scaler loaded.
Expected features: ['runs_left' 'balls_left' 'wickets_remaining' 'total_run_x' 'crr' 'rrr'
 'batting_team_Delhi Capitals' 'batting_team_Gujarat Titans'
 'batting_team_Kolkata Knight Riders' 'batting_team_Lucknow Super Giants'
 'batting_team_Mumbai Indians' 'batting_team_Punjab Kings'
 'batting_team_Rajasthan Royals'
 'batting_team_Royal Challengers Bangalore'
 'batting_team_Sunrisers Hyderabad' 'bowling_team_Delhi Capitals'
 'bowling_team_Gujarat Titans' 'bowling_team_Kolkata Knight Riders'
 'bowling_team_Lucknow Super Giants' 'bowling_team_Mumbai Indians'
 'bowling_team_Punjab Kings' 'bowling_team_Rajasthan Royals'
 'bowling_team_Royal Challengers Bangalore'
 'bowling_team_Sunrisers Hyderabad' 'city_Bengaluru' 'city_Chennai'
 'city_Delhi' 'city_Dharamsala' 'city_Guwahati' 'city_Hyderabad'
 'city_Jaipur' 'city_Kolkata' '

Runs Left (0-300): 20
Balls Left (0-120): 6
Wickets Remaining (0-10): 5
Target Score (0-300): 188

Current Run Rate: 8.84
Required Run Rate: 20.00


Teams: ['Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians', 'Chennai Super Kings', 'Royal Challengers Bangalore', 'Sunrisers Hyderabad', 'Punjab Kings', 'Rajasthan Royals']
Batting Team: Delhi Capitals
Bowling Team: Mumbai Indians
Cities: ['Bengaluru', 'Chennai', 'Delhi', 'Dharamsala', 'Guwahati', 'Mumbai', 'Kolkata', 'Hyderabad', 'Jaipur', 'Mullanpur/New Chandigarh', 'Visakhapatnam']
City: Delhi


Match Summary: Delhi Capitals vs Mumbai Indians at Delhi
Target: 188.0, Score: 168.0/5, Overs: 1.0/20


**3. Predictions using XGBoost Predictor**

In [ ]:
import pandas as pd
import numpy as np
import joblib
from google.colab import drive
from IPython.display import display, HTML
import os

# Mount Google Drive
drive.mount('/content/drive')

# Apply CSS styling
def apply_css():
    display(HTML("""
    <style>
        .main-header { color: #0066cc; text-align: center; font-size: 32px; font-weight: bold; margin-bottom: 20px; }
        .sub-header { color: #0066cc; font-size: 20px; font-weight: bold; margin: 20px 0 10px; }
        .prediction-box { background-color: #e6f0ff; border-radius: 10px; padding: 15px; margin: 10px 0; }
        .prediction-result { font-size: 20px; font-weight: bold; color: #006600; text-align: center; }
    </style>
    """))

# Load model and scaler
drive_path = '/content/drive/MyDrive/IPL match winner models'
model_path = os.path.join(drive_path, 'xgboost_model.pkl')
scaler_path = os.path.join(drive_path, 'scaler.joblib')

if os.path.exists(model_path) and os.path.exists(scaler_path):
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print("XGBoost model and scaler loaded.")
    print("Expected features:", scaler.feature_names_in_)
else:
    print("Model or scaler not found. Please train the model first.")
    model, scaler = None, None

# Define teams and cities
IPL_TEAMS = ['Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants',
             'Mumbai Indians', 'Chennai Super Kings', 'Royal Challengers Bangalore',
             'Sunrisers Hyderabad', 'Punjab Kings', 'Rajasthan Royals']
IPL_CITIES = ['Bengaluru', 'Chennai', 'Delhi', 'Dharamsala', 'Guwahati', 'Mumbai', 'Kolkata',
              'Hyderabad', 'Jaipur', 'Mullanpur/New Chandigarh', 'Visakhapatnam']

# Prepare input data
def prepare_input_data(runs_left, balls_left, wickets_remaining, total_run_x, crr, rrr, batting_team, bowling_team, city, scaler):
    data = {'runs_left': [runs_left], 'balls_left': [balls_left], 'wickets_remaining': [wickets_remaining],
            'total_run_x': [total_run_x], 'crr': [crr], 'rrr': [rrr]}
    input_df = pd.DataFrame(data)

    for team in IPL_TEAMS:
        input_df[f'batting_team_{team}'] = [1 if team == batting_team else 0]
        input_df[f'bowling_team_{team}'] = [1 if team == bowling_team else 0]
    for city_name in IPL_CITIES:
        input_df[f'city_{city_name}'] = [1 if city_name == city else 0]

    expected_features = scaler.feature_names_in_
    missing_features = [feat for feat in expected_features if feat not in input_df.columns]
    for feat in missing_features:
        input_df[feat] = 0
    input_df = input_df[expected_features]
    return input_df

# Main interface
def xgboost_interface():
    apply_css()
    display(HTML('<h1 class="main-header">XGBoost IPL Predictor</h1>'))
    if not model or not scaler:
        print("Cannot proceed without model and scaler.")
        return

    # Inputs
    display(HTML('<h2 class="sub-header">Match Situation</h2>'))
    runs_left = float(input("Runs Left (0-300): ") or 100)
    balls_left = float(input("Balls Left (0-120): ") or 60)
    wickets_remaining = int(input("Wickets Remaining (0-10): ") or 7)
    total_run_x = float(input("Target Score (0-300): ") or 180)

    current_score = total_run_x - runs_left
    overs_completed = 20 - (balls_left / 6)
    crr = current_score / overs_completed if overs_completed > 0 else 0
    rrr = (runs_left * 6) / balls_left if balls_left > 0 else 0
    print(f"\nCurrent Run Rate: {crr:.2f}")
    print(f"Required Run Rate: {rrr:.2f}")

    display(HTML('<h2 class="sub-header">Select Teams and City</h2>'))
    print(f"Teams: {IPL_TEAMS}")
    batting_team = input("Batting Team: ") or IPL_TEAMS[0]
    bowling_team = input("Bowling Team: ") or IPL_TEAMS[1]
    print(f"Cities: {IPL_CITIES}")
    city = input("City: ") or IPL_CITIES[0]

    if batting_team not in IPL_TEAMS or bowling_team not in IPL_TEAMS or city not in IPL_CITIES:
        print("Invalid input. Using defaults.")
        batting_team, bowling_team, city = IPL_TEAMS[0], IPL_TEAMS[1], IPL_CITIES[0]
    if batting_team == bowling_team:
        bowling_team = IPL_TEAMS[1] if batting_team == IPL_TEAMS[0] else IPL_TEAMS[0]

    # Predict
    input_data = prepare_input_data(runs_left, balls_left, wickets_remaining, total_run_x, crr, rrr, batting_team, bowling_team, city, scaler)
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]
    probability = model.predict_proba(input_scaled)[0][1] if prediction == 1 else model.predict_proba(input_scaled)[0][0]

    # Display result
    display(HTML('<div class="prediction-box">'))
    if prediction == 1:
        display(HTML(f'<p class="prediction-result">{batting_team} wins with {probability*100:.2f}% probability</p>'))
    else:
        display(HTML(f'<p class="prediction-result">{bowling_team} wins with {(1-probability)*100:.2f}% probability</p>'))
    print(f"Match Summary: {batting_team} vs {bowling_team} at {city}")
    print(f"Target: {total_run_x}, Score: {current_score}/{10-wickets_remaining}, Overs: {20-overs_completed:.1f}/20")
    display(HTML('</div>'))

if __name__ == '__main__':
    xgboost_interface()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
XGBoost model and scaler loaded.
Expected features: ['runs_left' 'balls_left' 'wickets_remaining' 'total_run_x' 'crr' 'rrr'
 'batting_team_Delhi Capitals' 'batting_team_Gujarat Titans'
 'batting_team_Kolkata Knight Riders' 'batting_team_Lucknow Super Giants'
 'batting_team_Mumbai Indians' 'batting_team_Punjab Kings'
 'batting_team_Rajasthan Royals'
 'batting_team_Royal Challengers Bangalore'
 'batting_team_Sunrisers Hyderabad' 'bowling_team_Delhi Capitals'
 'bowling_team_Gujarat Titans' 'bowling_team_Kolkata Knight Riders'
 'bowling_team_Lucknow Super Giants' 'bowling_team_Mumbai Indians'
 'bowling_team_Punjab Kings' 'bowling_team_Rajasthan Royals'
 'bowling_team_Royal Challengers Bangalore'
 'bowling_team_Sunrisers Hyderabad' 'city_Bengaluru' 'city_Chennai'
 'city_Delhi' 'city_Dharamsala' 'city_Guwahati' 'city_Hyderabad'
 'city_Jaipur' 'city_Kolkata' 'city_L

Runs Left (0-300): 114
Balls Left (0-120): 60
Wickets Remaining (0-10): 8
Target Score (0-300): 200

Current Run Rate: 8.60
Required Run Rate: 11.40


Teams: ['Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians', 'Chennai Super Kings', 'Royal Challengers Bangalore', 'Sunrisers Hyderabad', 'Punjab Kings', 'Rajasthan Royals']
Batting Team: Mumbai Indians
Bowling Team: Kolkata Knight Riders
Cities: ['Bengaluru', 'Chennai', 'Delhi', 'Dharamsala', 'Guwahati', 'Mumbai', 'Kolkata', 'Hyderabad', 'Jaipur', 'Mullanpur/New Chandigarh', 'Visakhapatnam']
City: Kolkata


Match Summary: Mumbai Indians vs Kolkata Knight Riders at Kolkata
Target: 200.0, Score: 86.0/2, Overs: 10.0/20


**5. Predictions using Logistic Regression**

In [ ]:
import pandas as pd
import numpy as np
import joblib
from google.colab import drive
from IPython.display import display, HTML
import os

# Mount Google Drive
drive.mount('/content/drive')

# Apply CSS styling
def apply_css():
    display(HTML("""
    <style>
        .main-header { color: #0066cc; text-align: center; font-size: 32px; font-weight: bold; margin-bottom: 20px; }
        .sub-header { color: #0066cc; font-size: 20px; font-weight: bold; margin: 20px 0 10px; }
        .prediction-box { background-color: #e6f0ff; border-radius: 10px; padding: 15px; margin: 10px 0; }
        .prediction-result { font-size: 20px; font-weight: bold; color: #006600; text-align: center; }
    </style>
    """))

# Load model and scaler
drive_path = '/content/drive/MyDrive/IPL match winner models'
model_path = os.path.join(drive_path, 'logistic_regression_model.pkl')
scaler_path = os.path.join(drive_path, 'scaler.joblib')

if os.path.exists(model_path) and os.path.exists(scaler_path):
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print("Logistic Regression model and scaler loaded.")
    print("Expected features:", scaler.feature_names_in_)
else:
    print("Model or scaler not found. Please train the model first.")
    model, scaler = None, None

# Define teams and cities
IPL_TEAMS = ['Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants',
             'Mumbai Indians', 'Chennai Super Kings', 'Royal Challengers Bangalore',
             'Sunrisers Hyderabad', 'Punjab Kings', 'Rajasthan Royals']
IPL_CITIES = ['Bengaluru', 'Chennai', 'Delhi', 'Dharamsala', 'Guwahati', 'Mumbai', 'Kolkata',
              'Hyderabad', 'Jaipur', 'Mullanpur/New Chandigarh', 'Visakhapatnam']

# Prepare input data
def prepare_input_data(runs_left, balls_left, wickets_remaining, total_run_x, crr, rrr, batting_team, bowling_team, city, scaler):
    data = {'runs_left': [runs_left], 'balls_left': [balls_left], 'wickets_remaining': [wickets_remaining],
            'total_run_x': [total_run_x], 'crr': [crr], 'rrr': [rrr]}
    input_df = pd.DataFrame(data)

    for team in IPL_TEAMS:
        input_df[f'batting_team_{team}'] = [1 if team == batting_team else 0]
        input_df[f'bowling_team_{team}'] = [1 if team == bowling_team else 0]
    for city_name in IPL_CITIES:
        input_df[f'city_{city_name}'] = [1 if city_name == city else 0]

    expected_features = scaler.feature_names_in_
    missing_features = [feat for feat in expected_features if feat not in input_df.columns]
    for feat in missing_features:
        input_df[feat] = 0
    input_df = input_df[expected_features]
    return input_df

# Main interface
def logistic_regression_interface():
    apply_css()
    display(HTML('<h1 class="main-header">Logistic Regression IPL Predictor</h1>'))
    if not model or not scaler:
        print("Cannot proceed without model and scaler.")
        return

    # Inputs
    display(HTML('<h2 class="sub-header">Match Situation</h2>'))
    runs_left = float(input("Runs Left (0-300): ") or 100)
    balls_left = float(input("Balls Left (0-120): ") or 60)
    wickets_remaining = int(input("Wickets Remaining (0-10): ") or 7)
    total_run_x = float(input("Target Score (0-300): ") or 180)

    current_score = total_run_x - runs_left
    overs_completed = 20 - (balls_left / 6)
    crr = current_score / overs_completed if overs_completed > 0 else 0
    rrr = (runs_left * 6) / balls_left if balls_left > 0 else 0
    print(f"\nCurrent Run Rate: {crr:.2f}")
    print(f"Required Run Rate: {rrr:.2f}")

    display(HTML('<h2 class="sub-header">Select Teams and City</h2>'))
    print(f"Teams: {IPL_TEAMS}")
    batting_team = input("Batting Team: ") or IPL_TEAMS[0]
    bowling_team = input("Bowling Team: ") or IPL_TEAMS[1]
    print(f"Cities: {IPL_CITIES}")
    city = input("City: ") or IPL_CITIES[0]

    if batting_team not in IPL_TEAMS or bowling_team not in IPL_TEAMS or city not in IPL_CITIES:
        print("Invalid input. Using defaults.")
        batting_team, bowling_team, city = IPL_TEAMS[0], IPL_TEAMS[1], IPL_CITIES[0]
    if batting_team == bowling_team:
        bowling_team = IPL_TEAMS[1] if batting_team == IPL_TEAMS[0] else IPL_TEAMS[0]

    # Predict
    input_data = prepare_input_data(runs_left, balls_left, wickets_remaining, total_run_x, crr, rrr, batting_team, bowling_team, city, scaler)
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]
    probability = model.predict_proba(input_scaled)[0][1] if prediction == 1 else model.predict_proba(input_scaled)[0][0]

    # Display result
    display(HTML('<div class="prediction-box">'))
    if prediction == 1:
        display(HTML(f'<p class="prediction-result">{batting_team} wins with {probability*100:.2f}% probability</p>'))
    else:
        display(HTML(f'<p class="prediction-result">{bowling_team} wins with {(1-probability)*100:.2f}% probability</p>'))
    print(f"Match Summary: {batting_team} vs {bowling_team} at {city}")
    print(f"Target: {total_run_x}, Score: {current_score}/{10-wickets_remaining}, Overs: {20-overs_completed:.1f}/20")
    display(HTML('</div>'))

if __name__ == '__main__':
    logistic_regression_interface()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Logistic Regression model and scaler loaded.
Expected features: ['runs_left' 'balls_left' 'wickets_remaining' 'total_run_x' 'crr' 'rrr'
 'batting_team_Delhi Capitals' 'batting_team_Gujarat Titans'
 'batting_team_Kolkata Knight Riders' 'batting_team_Lucknow Super Giants'
 'batting_team_Mumbai Indians' 'batting_team_Punjab Kings'
 'batting_team_Rajasthan Royals'
 'batting_team_Royal Challengers Bangalore'
 'batting_team_Sunrisers Hyderabad' 'bowling_team_Delhi Capitals'
 'bowling_team_Gujarat Titans' 'bowling_team_Kolkata Knight Riders'
 'bowling_team_Lucknow Super Giants' 'bowling_team_Mumbai Indians'
 'bowling_team_Punjab Kings' 'bowling_team_Rajasthan Royals'
 'bowling_team_Royal Challengers Bangalore'
 'bowling_team_Sunrisers Hyderabad' 'city_Bengaluru' 'city_Chennai'
 'city_Delhi' 'city_Dharamsala' 'city_Guwahati' 'city_Hyderabad'
 'city_Jaipur' 'city_Kolk

Runs Left (0-300): 144
Balls Left (0-120): 78
Wickets Remaining (0-10): 9
Target Score (0-300): 188

Current Run Rate: 6.29
Required Run Rate: 11.08


Teams: ['Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians', 'Chennai Super Kings', 'Royal Challengers Bangalore', 'Sunrisers Hyderabad', 'Punjab Kings', 'Rajasthan Royals']
Batting Team: Chennai Super Kings
Bowling Team: Gujrat Titans
Cities: ['Bengaluru', 'Chennai', 'Delhi', 'Dharamsala', 'Guwahati', 'Mumbai', 'Kolkata', 'Hyderabad', 'Jaipur', 'Mullanpur/New Chandigarh', 'Visakhapatnam']
City: Chennai
Invalid input. Using defaults.


Match Summary: Delhi Capitals vs Gujarat Titans at Bengaluru
Target: 188.0, Score: 44.0/1, Overs: 13.0/20


In [ ]:
import pandas as pd
import numpy as np
import joblib
from google.colab import drive
from IPython.display import display, HTML
import os

# Mount Google Drive
drive.mount('/content/drive')

# Apply CSS styling
def apply_css():
    display(HTML("""
    <style>
        .main-header { color: #0066cc; text-align: center; font-size: 32px; font-weight: bold; margin-bottom: 20px; }
        .sub-header { color: #0066cc; font-size: 20px; font-weight: bold; margin: 20px 0 10px; }
        .prediction-box { background-color: #e6f0ff; border-radius: 10px; padding: 15px; margin: 10px 0; }
        .prediction-result { font-size: 20px; font-weight: bold; color: #006600; text-align: center; }
    </style>
    """))

# Load model and scaler
drive_path = '/content/drive/MyDrive/IPL match winner models'
model_path = os.path.join(drive_path, 'svm_model.pkl')
scaler_path = os.path.join(drive_path, 'scaler.joblib')

if os.path.exists(model_path) and os.path.exists(scaler_path):
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print("SVM model and scaler loaded.")
    print("Expected features:", scaler.feature_names_in_)
else:
    print("Model or scaler not found. Please train the model first.")
    model, scaler = None, None

# Define teams and cities
IPL_TEAMS = ['Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants',
             'Mumbai Indians', 'Chennai Super Kings', 'Royal Challengers Bangalore',
             'Sunrisers Hyderabad', 'Punjab Kings', 'Rajasthan Royals']
IPL_CITIES = ['Bengaluru', 'Chennai', 'Delhi', 'Dharamsala', 'Guwahati', 'Mumbai', 'Kolkata',
              'Hyderabad', 'Jaipur', 'Mullanpur/New Chandigarh', 'Visakhapatnam']

# Prepare input data
def prepare_input_data(runs_left, balls_left, wickets_remaining, total_run_x, crr, rrr, batting_team, bowling_team, city, scaler):
    data = {'runs_left': [runs_left], 'balls_left': [balls_left], 'wickets_remaining': [wickets_remaining],
            'total_run_x': [total_run_x], 'crr': [crr], 'rrr': [rrr]}
    input_df = pd.DataFrame(data)

    for team in IPL_TEAMS:
        input_df[f'batting_team_{team}'] = [1 if team == batting_team else 0]
        input_df[f'bowling_team_{team}'] = [1 if team == bowling_team else 0]
    for city_name in IPL_CITIES:
        input_df[f'city_{city_name}'] = [1 if city_name == city else 0]

    expected_features = scaler.feature_names_in_
    missing_features = [feat for feat in expected_features if feat not in input_df.columns]
    for feat in missing_features:
        input_df[feat] = 0
    input_df = input_df[expected_features]
    return input_df

# Main interface
def svm_interface():
    apply_css()
    display(HTML('<h1 class="main-header">SVM IPL Predictor</h1>'))
    if not model or not scaler:
        print("Cannot proceed without model and scaler.")
        return

    # Inputs
    display(HTML('<h2 class="sub-header">Match Situation</h2>'))
    runs_left = float(input("Runs Left (0-300): ") or 100)
    balls_left = float(input("Balls Left (0-120): ") or 60)
    wickets_remaining = int(input("Wickets Remaining (0-10): ") or 7)
    total_run_x = float(input("Target Score (0-300): ") or 180)

    current_score = total_run_x - runs_left
    overs_completed = 20 - (balls_left / 6)
    crr = current_score / overs_completed if overs_completed > 0 else 0
    rrr = (runs_left * 6) / balls_left if balls_left > 0 else 0
    print(f"\nCurrent Run Rate: {crr:.2f}")
    print(f"Required Run Rate: {rrr:.2f}")

    display(HTML('<h2 class="sub-header">Select Teams and City</h2>'))
    print(f"Teams: {IPL_TEAMS}")
    batting_team = input("Batting Team: ") or IPL_TEAMS[0]
    bowling_team = input("Bowling Team: ") or IPL_TEAMS[1]
    print(f"Cities: {IPL_CITIES}")
    city = input("City: ") or IPL_CITIES[0]

    if batting_team not in IPL_TEAMS or bowling_team not in IPL_TEAMS or city not in IPL_CITIES:
        print("Invalid input. Using defaults.")
        batting_team, bowling_team, city = IPL_TEAMS[0], IPL_TEAMS[1], IPL_CITIES[0]
    if batting_team == bowling_team:
        bowling_team = IPL_TEAMS[1] if batting_team == IPL_TEAMS[0] else IPL_TEAMS[0]

    # Predict
    input_data = prepare_input_data(runs_left, balls_left, wickets_remaining, total_run_x, crr, rrr, batting_team, bowling_team, city, scaler)
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]
    probability = model.predict_proba(input_scaled)[0][1] if prediction == 1 else model.predict_proba(input_scaled)[0][0]

    # Display result
    display(HTML('<div class="prediction-box">'))
    if prediction == 1:
        display(HTML(f'<p class="prediction-result">{batting_team} wins with {probability*100:.2f}% probability</p>'))
    else:
        display(HTML(f'<p class="prediction-result">{bowling_team} wins with {(1-probability)*100:.2f}% probability</p>'))
    print(f"Match Summary: {batting_team} vs {bowling_team} at {city}")
    print(f"Target: {total_run_x}, Score: {current_score}/{10-wickets_remaining}, Overs: {20-overs_completed:.1f}/20")
    display(HTML('</div>'))

if __name__ == '__main__':
    svm_interface()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
SVM model and scaler loaded.
Expected features: ['runs_left' 'balls_left' 'wickets_remaining' 'total_run_x' 'crr' 'rrr'
 'batting_team_Delhi Capitals' 'batting_team_Gujarat Titans'
 'batting_team_Kolkata Knight Riders' 'batting_team_Lucknow Super Giants'
 'batting_team_Mumbai Indians' 'batting_team_Punjab Kings'
 'batting_team_Rajasthan Royals'
 'batting_team_Royal Challengers Bangalore'
 'batting_team_Sunrisers Hyderabad' 'bowling_team_Delhi Capitals'
 'bowling_team_Gujarat Titans' 'bowling_team_Kolkata Knight Riders'
 'bowling_team_Lucknow Super Giants' 'bowling_team_Mumbai Indians'
 'bowling_team_Punjab Kings' 'bowling_team_Rajasthan Royals'
 'bowling_team_Royal Challengers Bangalore'
 'bowling_team_Sunrisers Hyderabad' 'city_Bengaluru' 'city_Chennai'
 'city_Delhi' 'city_Dharamsala' 'city_Guwahati' 'city_Hyderabad'
 'city_Jaipur' 'city_Kolkata' 'city_Luckn

Runs Left (0-300): 100
Balls Left (0-120): 55
Wickets Remaining (0-10): 5
Target Score (0-300): 188

Current Run Rate: 8.12
Required Run Rate: 10.91


Teams: ['Delhi Capitals', 'Gujarat Titans', 'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians', 'Chennai Super Kings', 'Royal Challengers Bangalore', 'Sunrisers Hyderabad', 'Punjab Kings', 'Rajasthan Royals']
Batting Team: Gujarat Titans
Bowling Team: Lucknow Super Giants
Cities: ['Bengaluru', 'Chennai', 'Delhi', 'Dharamsala', 'Guwahati', 'Mumbai', 'Kolkata', 'Hyderabad', 'Jaipur', 'Mullanpur/New Chandigarh', 'Visakhapatnam']
City: Kolkata


Match Summary: Gujarat Titans vs Lucknow Super Giants at Kolkata
Target: 188.0, Score: 88.0/5, Overs: 9.2/20
